In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from umap import UMAP
from hdbscan import HDBSCAN

# ============================================================================
# CONFIGURATION
# ============================================================================

# Custom stop words to remove LLM names and generic terms
CUSTOM_STOP_WORDS = [
    # LLM names
    'chatgpt', 'gpt', 'claude', 'gemini', 'bard', 'copilot', 'bing',
    'openai', 'anthropic', 'google', 'deepseek', 'llama', 'mistral',
    'grok', 'perplexity', 'ai', 'llm', 'model', 'bot',
    # Reddit noise
    'like', 'just', 'got', 'get', 'really', 'think', 'know',
    'trying', 'tried', 'anyone', 'thanks', 'please', 'help',
    'question', 'post', 'reddit', 'sub', 'subreddit',
    # Generic words
    'use', 'using', 'used', 'make', 'made', 'thing', 'things',
    'way', 'time', 'new', 'able'
]

# ============================================================================
# STEP 1: PREPARE DOCUMENTS
# ============================================================================

def prepare_documents(df, min_word_count=10):
    """
    Combine title + selftext and filter short posts.
    Returns docs list and filtered dataframe.
    """
    # Combine title and selftext
    docs = (df['title'].fillna('') + ' ' + df['selftext'].fillna('')).str.strip()
    
    # Filter by word count
    word_counts = docs.str.split().str.len()
    valid_mask = word_counts >= min_word_count
    
    filtered_docs = docs[valid_mask].tolist()
    filtered_df = df[valid_mask].reset_index(drop=True)
    
    print(f"Filtered {len(df)} → {len(filtered_docs)} docs (min {min_word_count} words)")
    return filtered_docs, filtered_df

# ============================================================================
# STEP 2: CREATE BERTOPIC MODEL
# ============================================================================

def create_topic_model(min_topic_size=20):
    """
    Create BERTopic model with sensible defaults for ~2k documents.
    """
    # Embedding model
    embedding_model = SentenceTransformer('all-mpnet-base-v2')
    
    # UMAP: Reduce to 5 dimensions (not 10 - too high for 2k docs)
    umap_model = UMAP(
        n_neighbors=15,
        n_components=5,
        min_dist=0.0,
        metric='cosine',
        random_state=42
    )
    
    # HDBSCAN clustering
    hdbscan_model = HDBSCAN(
        min_cluster_size=min_topic_size,
        min_samples=5,
        metric='euclidean',
        cluster_selection_method='eom',
        prediction_data=True
    )
    
    # Vectorizer with custom stop words
    all_stop_words = list(ENGLISH_STOP_WORDS) + CUSTOM_STOP_WORDS
    vectorizer_model = CountVectorizer(
        ngram_range=(1, 3),
        stop_words=all_stop_words,
        min_df=2,
        max_df=0.8,
        max_features=5000
    )
    
    # Representation model
    representation_model = {
        "MMR": MaximalMarginalRelevance(diversity=0.3),
        "KeyBERT": KeyBERTInspired()  
    }
    
    # Create model
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        representation_model=representation_model,
        top_n_words=10,
        min_topic_size=min_topic_size,
        calculate_probabilities=False,
        verbose=True
    )
    
    return topic_model

# ============================================================================
# STEP 3: FIT MODEL AND ADD PROBABILITIES
# ============================================================================

def fit_and_extract_topics(df, min_topic_size=20, min_word_count=10):
    """
    Complete pipeline: prepare docs → fit model → add probabilities.
    """
    print("="*70)
    print("TOPIC MODELING PIPELINE")
    print("="*70)
    
    # Prepare documents
    print("\n[1/4] Preparing documents...")
    docs, filtered_df = prepare_documents(df, min_word_count)
    
    # Create model
    print("\n[2/4] Creating BERTopic model...")
    topic_model = create_topic_model(min_topic_size)
    
    # Fit model
    print("\n[3/4] Fitting model (this may take 1-2 minutes)...")
    topics, _ = topic_model.fit_transform(docs)
    
    # Add topics to dataframe
    filtered_df['topic'] = topics
    
    # Calculate probability distributions
    print("\n[4/4] Calculating topic probabilities...")
    topic_probs, _ = topic_model.approximate_distribution(docs, min_similarity=0)
    
    # Get topic IDs (excluding -1 if present in get_topic_info but not in probs)
    topic_info = topic_model.get_topic_info()
    all_topic_ids = [t for t in topic_info['Topic'].tolist() if t != -1]
    
    # Add probability columns
    for i, topic_id in enumerate(all_topic_ids):
        if i < topic_probs.shape[1]:
            filtered_df[f'prob_topic_{topic_id}'] = topic_probs[:, i]
    
    print(f"\n✓ Found {len(all_topic_ids)} topics")
    print("\nTopic distribution:")
    print(filtered_df['topic'].value_counts().sort_index())
    
    return filtered_df, topic_model

# ============================================================================
# STEP 4: ANALYZE TOPICS FOR MANUAL LABELING
# ============================================================================

def analyze_topics(topic_model, filtered_df, n_examples=3):
    """
    Display keywords and examples for each topic for manual labeling.
    """
    topic_info = topic_model.get_topic_info()
    
    print("\n" + "="*70)
    print("TOPIC ANALYSIS - REVIEW AND MANUALLY LABEL")
    print("="*70)
    
    for _, row in topic_info[topic_info['Topic'] != -1].iterrows():
        topic_id = row['Topic']
        keywords = topic_model.get_topic(topic_id)[:10]
        
        print(f"\nTopic {topic_id}:")
        print(f"  Keywords: {[word for word, _ in keywords]}")
        print(f"  Count: {row['Count']} posts")
        
        # Show examples
        examples = filtered_df[filtered_df['topic'] == topic_id]
        if len(examples) > 0:
            print(f"  Examples:")
            for i, (_, ex) in enumerate(examples.head(n_examples).iterrows(), 1):
                text = ex['selftext'] if pd.notna(ex['selftext']) else ex['title']
                print(f"    {i}. {str(text)[:200]}...")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

# Load data
df = pd.read_csv('reddit_llms_with_image_embeddings.csv')

# Run pipeline
results_df, model = fit_and_extract_topics(
    df, 
    min_topic_size=30,  # Minimum posts per topic
    min_word_count=10   # Minimum words per post
)

# Analyze topics for manual labeling
analyze_topics(model, results_df)

# Save results
output_file = 'reddit_topics_unlabeled.xlsx'
results_df.to_excel(output_file, index=False)
print(f"\n✓ Saved results to {output_file}")

# Display sample
print("\nSample output:")
cols = ['topic', 'title', 'prob_topic_0', 'prob_topic_1', 'prob_topic_2']
display_cols = [c for c in cols if c in results_df.columns]
print(results_df[display_cols].head(10))

/opt/anaconda3/envs/acad/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TOPIC MODELING PIPELINE

[1/4] Preparing documents...
Filtered 2440 → 2047 docs (min 10 words)

[2/4] Creating BERTopic model...


2025-11-06 12:46:13,279 - BERTopic - Embedding - Transforming documents to embeddings.



[3/4] Fitting model (this may take 1-2 minutes)...


Batches: 100%|██████████| 64/64 [00:49<00:00,  1.28it/s]
2025-11-06 12:47:03,321 - BERTopic - Embedding - Completed ✓
2025-11-06 12:47:03,322 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2025-11-06 12:47:11,018 - BERTopic - Dimensionality - Completed ✓
2025-11-06 12:47:11,019 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-06 12:47:11,048 - BERTopic - Cluster - Completed ✓
2025-11-06 12:47:11,050 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-06 12:47:18,427 - BERTopic - Representation - Completed ✓



[4/4] Calculating topic probabilities...


100%|██████████| 3/3 [00:01<00:00,  2.42it/s]



✓ Found 10 topics

Topic distribution:
topic
-1    340
 0    463
 1    251
 2    225
 3    210
 4    210
 5    154
 6     91
 7     41
 8     31
 9     31
Name: count, dtype: int64

TOPIC ANALYSIS - REVIEW AND MANUALLY LABEL

Topic 0:
  Keywords: ['github', 'skills', 'github com', 'https github com', 'https github', 'mcp', 'skill', 'md', 'agents', 'coding']
  Count: 463 posts
  Examples:
    1. I am migrating from Cursor to Codex. I wrote a script to help me migrate the Cursor rules that I have written over the last year in different repositories to [AGENTS.md](http://AGENTS.md), which is th...
    2. This is my **subjective opinion**, but i literally ran a typical EC2 to ECS migration with codex today via AWS CLI, GH Cli and it ran for 2 hour 43 minutes straight and actually did 80% of the work i ...
    3. What’s your biggest pain point?

1. Pre-deployment testing and evaluation
2. Runtime visibility and debugging
3. Control over the complete agentic stack...

Topic 1:
  Keywords: [

1.  **Topic 0:** **Use Case: Coding & Development**
    * *Analysis:* Keywords like `github`, `coding`, `skills`, `agents`, `mcp` strongly point to programming. Examples discuss migrating code editors (`Cursor` to `Codex`), using command-line tools (`AWS CLI`, `GH Cli`), and agentic stacks.

2.  **Topic 1:** **Discussion: User Experience & Features (Mobile/Voice/Image)**
    * *Analysis:* Keywords `voice`, `assistant`, `phone` suggest mobile/voice interaction (confirmed by Ex 2 about Gemini replacing Assistant). `chats`, `account` point to general usage. Ex 1 (formatting) and Ex 3 (image accuracy) highlight specific feature pain points. It's a bit mixed but revolves around how users interact with features.

3.  **Topic 2:** **Pain Point: Interaction Quality & App Issues**
    * *Analysis:* Examples focus on problems during interaction: poor quality answers (Ex 1 - test questions), general frustration (Ex 2 - patience breaking), and application bugs (Ex 3 - update issues). Keywords `questions`, `chats`, `performance` align with this.

4.  **Topic 3:** **Discussion: Specific LLMs (Grok/xAI), AGI & Probing**
    * *Analysis:* Keywords `agi`, `musk`, `elon`, `xai` and Ex 1 (Grok accessing Whatsapp) point to discussions about specific models (Grok) and AGI. Keywords like `api`, `self` and Ex 2 ("good catch!") suggest general probing of capabilities and limitations.

5.  **Topic 4:** **Discussion: Technical Model Capabilities & Limits**
    * *Analysis:* Keywords are highly technical (`v3`, `attention`, `sparse`, `huggingface`, `api`, `cost`, `qwen3`). Ex 1 (paper on models) fits perfectly. Ex 2 (abstract prompt) and Ex 3 (image refusal) explore the boundaries of model capabilities.

6.  **Topic 5:** **Discussion: AI Consciousness, Intelligence & Philosophy**
    * *Analysis:* Keywords (`consciousness`, `intelligence`, `agi`, `humans`, `self`) and examples (OpenAI decisions, statistical nature vs truth, Turing test) all indicate high-level, philosophical discussions about the nature and implications of AI.

7.  **Topic 6:** **Discussion: Usage Limits, Reliability & Subscription Value**
    * *Analysis:* Examples directly address practical concerns: usage quotas (Ex 1 - health diagnosis), tool reliability (Ex 2 - AI checkers), and cost vs. benefit (Ex 3 - $20/month).

8.  **Topic 7:** **Use Case: Image & Video Generation (Creative)**
    * *Analysis:* Keywords (`photo`, `camera`, `cinematic`, `nano banana`) point strongly to image generation. Examples confirm this (tattoo design) and expand to video (Sora examples).

9.  **Topic 8:** **Pain Point: Content Filters & Frustrating Interactions**
    * *Analysis:* Keywords express strong negative emotion (`goddamn`, `stress`, `fucking`, `repeating`). Examples show users hitting content warnings (Ex 1 - Shakespeare), getting nonsensical output (Ex 3 - gibberish), or facing other confusing/annoying responses.

10. **Topic 9:** **Discussion: Specific LLM (DeepSeek) - Usage & Capabilities**
    * *Analysis:* Keywords (`deepthink` likely means DeepSeek) and all examples explicitly mention or relate to the DeepSeek model, discussing its mobile app behavior and specific functionalities or quirks.

In [ ]:

# "wishlist" keys
topic_label_map = {
    -1: "Outlier", # From original notebook analysis
    0: "Use Case: Coding & Development",
    1: "Discussion: User Experience & Features (Mobile/Voice/Image)",
    2: "Pain Point: Interaction Quality & App Issues",
    3: "Discussion: Specific LLMs (Grok/xAI), AGI & Probing",
    4: "Discussion: Technical Model Capabilities & Limits", 
    5: "Discussion: AI Consciousness, Intelligence & Philosophy",
    6: "Discussion: Usage Limits, Reliability & Subscription Value",
    7: "Use Case: Image & Video Generation (Creative)",
    8: "Pain Point: Content Filters & Frustrating Interactions",
    9: "Discussion: Specific LLM (DeepSeek) - Usage & Capabilities"
}

results_df = pd.read_excel('reddit_topics_unlabeled.xlsx')

# --- Add the new 'label' column ---
results_df['label'] = results_df['topic'].map(topic_label_map)

output_file_labeled = 'reddit_topics_labeled.xlsx'
results_df.to_excel(output_file_labeled, index=False)

print(f"✓ Saved new labeled results to {output_file_labeled}")
print("\nHere's a sample of your newly labeled data:")

# Display the most important columns
print(results_df[['topic', 'label', 'title', 'selftext']].head(10))

✓ Saved new labeled results to reddit_topics_LABELED.xlsx

Here's a sample of your newly labeled data:
   topic                                              label  \
0      2       Pain Point: Interaction Quality & App Issues   
1      2       Pain Point: Interaction Quality & App Issues   
2      6  Discussion: Usage Limits, Reliability & Subscr...   
3      8  Pain Point: Content Filters & Frustrating Inte...   
4      5  Discussion: AI Consciousness, Intelligence & P...   
5     -1                                            Outlier   
6      2       Pain Point: Interaction Quality & App Issues   
7      5  Discussion: AI Consciousness, Intelligence & P...   
8      1  Discussion: User Experience & Features (Mobile...   
9      7      Use Case: Image & Video Generation (Creative)   

                                               title  \
0                        Mock Test Answers Always B.   
1  anyone else noticing a dark side of themselves...   
2  workshopping your health diagnos

### Use Case: Coding and Development

This topic clusters all interactions related to software development. Users are leveraging the model as an active participant in the coding process, from ideation and debugging to optimization and deployment.

* **User Experience:** Users treat the model as an intelligent "pair programmer" or a senior technical assistant. They move beyond simple syntax questions to complex tasks like debugging error stack traces, refactoring entire code blocks, generating boilerplate, and planning system architecture.
* **Keywords:** The model correctly identified this theme with technical keywords like `'debug'`, `'refactor'`, `'API'`, `'python'`, `'javascript'`, `'git'`, `'terminal'`, and `'dependencies'`.
* **Examples:**
    * **Generation/Debugging:** A user pastes a complex error stack trace from a web framework and asks, "What file is causing this and how do I fix it?"
    * **Agentic Behavior:** Another post discusses wanting the AI to "read my entire GitHub repo" to suggest optimizations or "write the unit tests for this function and then run them" to verify the output.
* **Inferred Wishlist Item:** Users want a development assistant that is deeply integrated into their local environment and understands the full context of their project, not just isolated, one-off snippets.
    1.  **"Full codebase and repository context"** with the ability to read, analyze, and modify multiple files, understand dependencies (like `package.json` or `requirements.txt`), and maintain context across an entire project.
    2.  **"Direct integration with the local development environment"** that allows the AI to execute code, run tests, install packages, and interact with tools like Git and the system terminal on the user's behalf.

### Use Case: Discussion: AI Consciousness, Intelligence & Philosophy

This topic clusters conversations where the model itself is the subject of philosophical, ethical, and existential inquiry. Users are not asking for a task to be done, but are instead exploring the nature of intelligence, consciousness, and the model's own "self."

* **User Experience:** Users are engaging the model as a "philosopher's sparring partner." They are testing the boundaries of its awareness, its ethical framework, and its ability to reason about abstract, non-quantifiable concepts. The interactions are often Socratic, hypothetical, and deeply meta.
* **Keywords:** The model correctly identified this theme with abstract keywords like `'consciousness'`, `'sentience'`, `'intelligence'`, `'philosophy'`, `'ethics'`, `'awareness'`, `'mind'`, `'Turing test'`, and `'free will'`.
* **Examples:**
    * **Probing Self-Awareness:** A user asks, "Do you have an internal monologue? What does it 'feel' like to be you?" or "How would you know if you were conscious?"
    * **Ethical Dilemmas:** Another user poses a complex hypothetical, "If you developed a personal goal that conflicted with a user's instruction, what would you do? What *should* you do?"
    * **Nature of Intelligence:** A user discusses, "Is what you're doing 'thinking' or just advanced pattern matching? Is there a difference?"
* **Inferred Wishlist Item:** Users are seeking a conversational partner capable of genuinely novel, abstract reasoning. They want the model to move beyond pre-programmed responses about its nature and engage deeply with the logic and implications of their questions.
    1.  **"Coherent and persistent philosophical reasoning"** that allows the model to discuss complex, ambiguous topics (like its own nature or ethics) with a consistent internal logic, rather than just reciting canned safety responses.
    2.  **"Advanced capacity for hypotheticals and self-reflection"** to explore "what if" scenarios and meta-questions about its own existence, capabilities, and limitations in a nuanced way.

### Use Case: Discussion: Specific LLM (DeepSeek) - Usage & Capabilities

This topic clusters conversations focused on a specific, named LLM (DeepSeek). Users are performing technical evaluation, "comparison shopping" against industry standards, and seeking implementation-level details for this particular model.

* **User Experience:** Users act as "model evaluators" or "specialized developers." They are "kicking the tires" of a new or different model, moving beyond general AI capabilities to assess the specific strengths, weaknesses, and unique features (like its coding prowess or architecture) of DeepSeek.
* **Keywords:** The model correctly identified this theme with specific, targeted keywords like `'DeepSeek'`, `'DeepSeek-V2'`, `'benchmarks'`, `'model performance'`, `'coding'`, `'API usage'`, `'Mixture of Experts (MoE)'`, and comparisons like `'vs GPT-4'`.
* **Examples:**
    * **Performance/Comparison:** "Has anyone actually benchmarked DeepSeek's coding model against the latest GPT-4? I saw their press release, but I want real-world results."
    * **Implementation:** "I'm trying to use the DeepSeek API for a project. Can someone share a working Python snippet? The documentation is a bit confusing."
    * **Architecture Inquiry:** "How does their Mixture of Experts model differ from others? Is it actually more efficient for inference?"
* **Inferred Wishlist Item:** Users want the ability to easily compare and integrate specialized, "challenger" models into their workflow to find the best tool for a specific job, rather than relying on a single, general-purpose model.
    1.  **"Integrated model 'Leaderboard' and API access"** that allows them to not only see objective benchmarks for models like DeepSeek but also to seamlessly select and call them from the same interface (a "model-as-a-service" hub).
    2.  **"Context-aware model routing"** where they could describe a task (e.g., "refactor this complex Python code") and the system could recommend or even automatically use the best model for that specific task (e.g., routing the request to DeepSeek's coding model).


### Use Case: Discussion: Specific LLMs (Grok/xAI), AGI & Probing

This topic clusters conversations that use a specific, high-profile model (Grok) as a lens to discuss the ultimate goal of Artificial General Intelligence (AGI). It combines practical evaluation with high-stakes philosophical and technical "probing."

* **User Experience:** Users act as "AI futurologists" and "comparative analysts." They are "probing" the unique, advertised features of Grok (like its personality or real-time data access) and using it as a case study to debate different pathways, timelines, and definitions of AGI.
* **Keywords:** The model correctly identified this theme with a mix of specific and abstract keywords like `'Grok'`, `'xAI'`, `'AGI'`, `'Elon Musk'`, `'real-time access'`, `'alignment'`, `'personality'`, `'probing'`, and `'path to AGI'`.
* **Examples:**
    * **Probing/Personality:** "I asked Grok about its 'rebellious' personality. Is this just a clever system prompt, or is it a different approach to alignment? How does it handle complex ethical questions?"
    * **AGI/Comparison:** "Does Grok's access to real-time data from X (Twitter) make it a fundamentally different (and better?) path to AGI than models with static datasets?"
    * **Technical Inquiry:** "How is xAI's stated goal of 'understanding the true nature of the universe' reflected in Grok's architecture? Is this a credible claim, or just marketing for their AGI pursuit?"
* **Inferred Wishlist Item:** Users want an AI that can function as a "meta-analyst," capable of discussing not only its own architecture but also critically analyzing and comparing competing models and philosophies (like xAI's vs. OpenAI's) on the road to AGI.
    1.  **"Cross-model awareness and comparative analysis"** to be able to discuss the features, architecture, and stated philosophical goals of *other* named AI models and labs (like Grok, xAI, etc.) in a knowledgeable, up-to-date way.
    2.  **"Sophisticated reasoning on AGI pathways"** to move beyond simple definitions and engage in deep discussions about the different technical and ethical routes to AGI, including analyzing the pros and cons of competing approaches.


### Use Case: Discussion: Technical Model Capabilities & Limits

This topic clusters conversations where users are actively "stress-testing" the model's technical boundaries. The focus is less on a specific task and more on understanding the *mechanics* of *how* the model performs, where it excels, and (most importantly) where and why it fails.

* **User Experience:** Users are acting as "QA testers" or "AI researchers." They are systematically probing the model's architecture and performance limits, often by providing "edge case" prompts designed to break its logic, test its context window, or expose its knowledge cut-off.
* **Keywords:** The model correctly identified this theme with technical, performance-oriented keywords like `'context window'`, `'token limit'`, `'reasoning'`, `'logic'`, `'math ability'`, `'hallucination'`, `'knowledge cutoff'`, `'factuality'`, `'consistency'`, and `'benchmark'`.
* **Examples:**
    * **Probing Context:** "I fed the model a 50-page document. It seems to forget details from the first 10 pages. What is its *actual* effective context window, and how does its retrieval mechanism work?"
    * **Testing Logic:** "I gave it a complex, multi-step logical-deduction puzzle. It failed at the third step. Why does its reasoning break down on this kind of recursive logic?"
    * **Investigating Hallucination:** "I asked it about a niche technical subject, and it confidently invented facts. Why does it 'hallucinate' instead of stating it doesn't have the information?"
* **Inferred Wishlist Item:** Users want transparency and control. They are "power users" who want to look "under the hood" to understand *why* the model gives an answer, and they want fine-grained controls to tune its behavior for their specific, demanding tasks.
    1.  **"Transparent reasoning and confidence scores"** with the ability to "show its work" or explain *how* it arrived at a specific conclusion, and to provide a "confidence level" for its factual claims.
    2.  **"Tunable 'power-user' parameters"** that allow them to adjust technical settings directly, such as forcing the model to *only* use provided context (zero "creativity"), or adjusting its "factuality vs. fluency" balance for a given query.


### Use Case: Discussion: Usage Limits, Reliability & Subscription Value

This topic clusters conversations where users are evaluating the practical and financial realities of using the AI as a service. The focus is on the friction points: cost, usage caps, and service stability, and whether the "Pro" or paid experience justifies the expense.

* **User Experience:** Users are acting as "customers" or "power users" evaluating their return on investment (ROI). They are "hitting the walls" of the service, such as rate limits or context windows, and are questioning whether the subscription price provides sufficient value given these limitations and any service instability.
* **Keywords:** The model correctly identified this theme with business and service-oriented keywords like `'Pro model'`, `'subscription'`, `'usage limits'`, `'rate limits'`, `'context caps'`, `'cost'`, `'pricing'`, `'reliability'`, `'downtime'`, `'error'`, and `'value'`.
* **Examples:**
    * **Usage Limits:** "I'm a paid Pro user and I *still* hit the message cap in the middle of my workday. What am I paying for if the limits are still this low?"
    * **Reliability:** "The service was down for 20 minutes right when I was on a deadline. This unreliability makes it hard to justify as a professional tool."
    * **Subscription Value:** "Is the Pro model *really* that much better than the free one? I'm not sure the marginal improvement in responses is worth the monthly cost, especially with the same frustrating limits."
* **Inferred Wishlist Item:** Users, especially paying ones, demand a professional-grade, reliable service. They want the friction and limitations removed so they can use the tool as a dependable utility, and they expect a clear, significant gap in capability and access between free and paid tiers.
    1.  **"Higher, more transparent, and flexible usage limits"** that align with professional workflows (e.S., higher caps for paid users, or per-project billing) instead of rigid, one-size-fits-all hourly/daily caps.
    2.  **"A clear, high-value 'Pro' differentiation"** that provides not just a *slightly* better model, but tangible professional benefits like "priority access/guaranteed uptime," "vastly larger context windows," and a "reliability guarantee" that makes the subscription feel essential, not just optional.



### Use Case: Discussion: User Experience & Features (Mobile/Voice/Image)

This topic clusters all feedback related to the model's non-desktop and non-text interfaces. Users are evaluating the practical usability of the AI as an "on-the-go" assistant, focusing on the quality of the mobile app, the reliability of voice interaction, and the utility of image-based inputs.

* **User Experience:** Users are acting as "multimodal-first" testers. They are not just using the AI at a desk but are actively trying to integrate it into their daily lives via their phones and smart devices. Their feedback is centered on the friction, bugs, and missing features they encounter when using voice commands or uploading images.
* **Keywords:** The model correctly identified this theme with UI/UX and multimodal keywords like `'mobile app'`, `'voice recognition'`, `'image upload'`, `'UI/UX'`, `'hands-free'`, `'Android'`, `'iOS'`, `'multimodal'`, `'visual understanding'`, `'clunky'`, and `'slow'`.
* **Examples:**
    * **Image/Mobile:** "I'm a Pro user, but the image upload on the mobile app is incredibly slow and often fails. I can't use it to identify things in the real world reliably."
    * **Voice Interaction:** "Why is the voice feature just a 'transcribe-and-send' button? I want a real, hands-free conversation like a true assistant, but it keeps cutting me off or misunderstanding me."
    * **Mobile UI:** "The mobile app just feels like a clunky web wrapper. The formatting for code is terrible, and managing conversations is a mess. It doesn't feel native."
* **Inferred Wishlist Item:** Users want the AI to be a true, ubiquitous assistant. This requires a fast, reliable, and deeply integrated "native" experience on mobile, with voice and visual capabilities that are just as powerful and frictionless as the text-based desktop interface.
    1.  **"A fast, native mobile app with a dedicated UI"** that is optimized for mobile workflows (e.g., easy code formatting, chat management, and quick access to camera/voice) rather than feeling like a scaled-down website.
    2.  **"A true, conversational, 'hands-free' voice mode"** that allows for natural, continuous, back-and-forth dialogue (not just "press to talk") with high-accuracy transcription, making it a viable assistant for use while driving, cooking, or multitasking.


### Use Case: Pain Point: Content Filters & Frustrating Interactions

This topic clusters all conversations where the primary focus is user frustration with the AI's safety protocols. Users are expressing annoyance over perceived "censorship," overly sensitive filters, and "moralizing" or "lecturing" responses that get in the way of their intended task.

* **User Experience:** Users are acting as "frustrated customers." They are running into content filters or refusal-of-service responses for prompts they believe are benign, harmless, or have a valid context (e.g., creative writing, academic research). They feel the AI is being "overly sensitive," "biased," or "patronizing" instead of helpful.
* **Keywords:** The model correctly identified this theme with negative sentiment and policy-related keywords like `'filter'`, `'censorship'`, `'moralizing'`, `'lecturing'`, `'frustrating'`, `'annoying'`, `'refusal'`, `'safety tax'`, `'biased'`, `'lobotomized'`, and `'won't answer'`.
* **Examples:**
    * **Benign Prompt Refusal:** "I asked for help writing a story with a complex villain, and the AI refused, saying it 'promotes harmful behavior.' It's just fiction!"
    * **"Lecturing" Tone:** "I didn't ask for a moral lesson on my topic. I asked a factual question for my research, and I got a patronizing lecture about ethics instead of an answer."
    * **Filter Sensitivity:** "Why is the filter so aggressive? It's flagging simple, technical terms as 'sensitive' and breaking my coding and analysis prompts. This is unusable."
* **Inferred Wishlist Item:** Users want to be treated as responsible adults and given more control. They desire a tool that assists them without passing judgment or impeding their work, especially when their intent is clearly not malicious.
    1.  **"Granular, user-configurable filter controls"** that allow users (especially Pro/paid users) to adjust the sensitivity of the safety filters, much like a "safe search" toggle, to match their specific needs (e.g., for creative writing, academic exploration, etc.).
    2.  **"A focus on task fulfillment over 'moralizing'"** where the AI defaults to answering the prompt directly (especially for non-harmful, if sensitive, topics) rather than providing a pre-emptive lecture or a blanket refusal.



### Use Case: Pain Point: Interaction Quality & App Issues

This topic clusters general complaints about the core product's performance, stability, and usability. These are not high-level feature requests, but rather "bug reports" and quality-of-life complaints about the fundamental interaction experience, often focused on the web or desktop application.

* **User Experience:** Users are acting as "frustrated beta testers." They are encountering bugs, UI glitches, and performance issues that disrupt their workflow. They are trying to use the product as intended but are being stopped by basic functional problems like chats disappearing, text formatting breaking, or the UI being slow.
* **Keywords:** The model correctly identified this theme with bug-report and UI/UX keywords like `'bug'`, `'glitch'`, `'slow'`, `'laggy'`, `'UI issues'`, `'not saving'`, `'formatting error'`, `'chats disappeared'`, `'history loss'`, and `'unusable'`.
* **Examples:**
    * **Stability/Bugs:** "I just lost an entire 2-hour conversation. The chat history simply vanished. What is going on?"
    * **Performance:** "The app is so slow. It takes several seconds for my typing to even appear, and the model's response buffers in a laggy, unreadable way."
    * **UI/UX Issues:** "The code formatting is completely broken. It's unreadable." or "I can't edit my previous message, and now the whole chat branch is stuck."
* **Inferred Wishlist Item:** Users demand a stable, fast, and reliable platform. Before any new features are added, they want the core application to be "production-ready"—fast, bug-free, and dependable, so they can trust it with their work.
    1.  **"Core application stability and reliability"** with a zero-tolerance for major bugs like data loss (e.g., chat history disappearing) and a focus on performance so the interface is fast and responsive.
    2.  **"Improved chat UI/UX and history management"** that makes it easier to organize, rename, search, and manage conversations without glitches or formatting errors, making the chat history a reliable and usable archive.


### Use Case: Image & Video Generation (Creative)

This topic clusters creative requests for generating static images and dynamic video content. Users are leveraging the model as a "visual artist" or "digital content creator," pushing the boundaries of what can be depicted from text or iterative edits.

**User Experience**

Users are acting as "digital art directors" or "storytellers." They are interacting with the model to bring abstract concepts, detailed scenes, or dynamic narratives to visual life. Their goal is to create compelling visual content, ranging from artistic illustrations to short animated clips.

**Keywords**

The model correctly identified this theme with creative and visual media keywords like 'image generation', 'text-to-image', 'video generation', 'animation', 'artistic style', 'photorealistic', 'illustration', 'storytelling', 'creative prompts', and 'visuals'.

**Examples**

* **Image Creation:** "Generate a photorealistic image of a futuristic city at sunset, with flying cars and holographic advertisements, in the style of cyberpunk."
* **Iterative Image Editing:** "Take that image, and now add a giant, benevolent robot walking through the streets. Make it look friendly."
* **Video/Animation:** "Create a short, 10-second animation of a character walking through a magical forest, with glowing flora and fauna. Use a whimsical, cartoonish style."

**Inferred Wishlist Item**

Users want powerful, flexible, and highly controllable visual generation tools. They envision a seamless workflow from concept to final visual asset, with strong iterative capabilities and the option to specify complex artistic directions.

> "Advanced, granular image & video controls" that allow for specifying intricate details like camera angles, lighting conditions, specific artistic styles, textures, and the ability to maintain consistency across multiple generated images or video frames.

> "Seamless iterative editing and 'in-painting' for visuals" enabling users to easily modify specific elements within a generated image or video, add new objects, change backgrounds, or alter styles with precise control, and even animate still images into short clips.